In [1]:
# playground for json generator
# Ruize Li, July 19 2021, Colby College


In [1]:
import pandas as pd
import numpy as np
import json
import os

In [2]:
# get current directory
cwd = os.getcwd()
print(cwd)

/Users/ruizeli/Documents/GitHub/cds_test/backend/convert_xls_to_json


In [9]:
# get all .xlsx files
onlyfiles = [f for f in os.listdir(cwd) if os.path.isfile(os.path.join(cwd, f))]
data_set = []
for f in onlyfiles:
    if f.endswith(".xlsx"):
        data_set.append(f)

In [10]:
data_set

['1950s  《人民画报》封面统计.xlsx', '1960s  《人民画报》封面统计.xlsx', '1970s 《人民画报》封面统计.xlsx']

In [12]:
# create pandas df
df = pd.concat((pd.read_excel(f) for f in data_set))

期刊日期           封面说明  \
0    1950年第1期            毛主席   
1    1950年第2期   中国第一位女拖拉机手梁军   
2    1950年第3期    和平保卫者“海陆空军”   
3    1950年第4期   庆祝中华人民共和国国庆节   
4    1950年第6期          丰盛的稻子   
..        ...            ...   
132   1979年8期      著名植物学家蔡希陶   
133   1979年9期          黄山玉屏楼   
134  1979年10期           版纳之花   
135  1979年11期  京剧《昭君出塞》中的王昭君   
136  1979年12期   全国第四届运动会比赛场上   

    职业 （工人，学生，国家领导，地方干部，儿童，国际友人，农民，革命烈士，军警/民兵，业务员，文娱，科研人员，服务行业，教育卫生)  \
0                                                 国家领导                 
1                                             农民/女拖拉机手                 
2                                                   军人                 
3                                                   群众                 
4                                                   农民                 
..                                                 ...                 
132                                               植物学家                 
133                                                NaN                 
134                                                NaN                 
135                                                 演员                 
136                                                运动员                 

    族裔（汉，少数民族）      性别 摄影作者                                                 备注  
0          NaN       男  陈正青  1950年7月，《人民画报》正式创刊，这本杂志从一开始就气派非凡——它采用了那个年代少见和珍...  
1          NaN       女  王纯德  1950年，新闻摄影局记者王纯德深入国营农场采访时拍摄了这张新中国第一位女拖拉机手的封面照片...  
2          NaN    男/集体  陈正青                                                NaN  
3          NaN  男/女/集体  陈正青                                                NaN  
4          NaN       男  吴宝基                                                NaN  
..         ...     ...  ...                                                ...  
132        NaN       男  NaN                                                NaN  
133        NaN     NaN  NaN                                                NaN  
134        NaN     NaN  NaN                                                NaN  
135        NaN       女  NaN                                                NaN  
136        NaN      集体  NaN                                                NaN  

[412 rows x 7 columns]

In [15]:
df.rename(columns=dict(zip(list(df.columns), range(7))), inplace=True)

df = df.fillna('')

# prepare keywords
df[7] = df.apply(lambda row: row[1] + row[2] + row[3] + row[4] + row[5] + row[6], axis=1)

df[8] = df[0].str.replace('年第', '_').str.replace('期', '').str.replace('年', '_')

# generate json
result = dict()

index = 0
for row in df.iterrows():
    date = row[1][8].split('_')
    if len(date) != 2:
        continue
    file_name = f'RMHB_{date[0]}_{date[1]}'
    keyword = row[1][7]
    result[file_name] = {'file_name': file_name,
                         'keyword': keyword,
                         'image_path': '',
                         'id' : index
                        }
    index += 1



# output json files
with open('result.json', 'w') as file:
    file.write(json.dumps(result))